## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
tf.enable_eager_execution()

### Collect Data

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Data/prices.csv')

### Check all columns in the dataset

In [30]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(['date','symbol'],axis=1)

In [32]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [33]:
data.shape

(851264, 5)

In [0]:
data = data.iloc[0:1000,]

In [35]:
data.shape

(1000, 5)

In [0]:
data.volume = data.volume/1000000

In [37]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [0]:
X = data.drop('volume',axis=1)
y = data.iloc[:,-1]

In [90]:
type(X)

pandas.core.frame.DataFrame

In [91]:
type(y)

pandas.core.series.Series

In [92]:
X.shape

(1000, 4)

In [93]:
y.shape

(1000,)

In [94]:
X.head()

,open,close,low,high
0,123.430000,125.839996,122.309998,126.250000
1,125.239998,119.980003,119.940002,125.540001
2,116.379997,114.949997,114.930000,119.739998
3,115.480003,116.620003,113.500000,117.440002
4,117.010002,114.970001,114.089996,117.330002


In [95]:
y.head()

0    2.1636
1    2.3864
2    2.4895
3    2.0063
4    1.4086
Name: volume, dtype: float64

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30 , random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [98]:
X_train.shape

(700, 4)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

In [0]:
y_train = np.array(y_train)

In [0]:
y_test = np.array(y_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

In [112]:
w

<tf.Tensor: id=5888476, shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>

In [0]:
xw_matmul = tf.matmul(X_train, w)

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [117]:
type(X_train)

numpy.ndarray

In [118]:
type(y_train)

numpy.ndarray

In [121]:
# Training loss at every iteration and Test loss at every 5th iteration

for i in range(100):
    
    w, b = train(X_train, y_train, w, b)
    print('Current train Loss on iteration', i, loss(y_train, prediction(X_train, w, b)).numpy())
    
    if(i%5==0):
      print('Current Test Loss on iteration', i, loss(y_test, prediction(X_test, w, b)).numpy())

Current train Loss on iteration 0 195.41153
Current Test Loss on iteration 0 240.90453
Current train Loss on iteration 1 195.41089
Current train Loss on iteration 2 195.41032
Current train Loss on iteration 3 195.40979
Current train Loss on iteration 4 195.40929
Current train Loss on iteration 5 195.40883
Current Test Loss on iteration 5 240.92453
Current train Loss on iteration 6 195.40843
Current train Loss on iteration 7 195.40804
Current train Loss on iteration 8 195.40767
Current train Loss on iteration 9 195.40735
Current train Loss on iteration 10 195.40706
Current Test Loss on iteration 10 240.94128
Current train Loss on iteration 11 195.4068
Current train Loss on iteration 12 195.4065
Current train Loss on iteration 13 195.40627
Current train Loss on iteration 14 195.40607
Current train Loss on iteration 15 195.40588
Current Test Loss on iteration 15 240.9552
Current train Loss on iteration 16 195.4057
Current train Loss on iteration 17 195.40552
Current train Loss on iteratio

### Get the shapes and values of W and b

In [124]:
w.shape

TensorShape([Dimension(4), Dimension(1)])

In [125]:
b.shape

TensorShape([Dimension(1)])

In [122]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1.3777771]
 [1.3827603]
 [1.3636918]
 [1.3945339]]
Bias:
 [2.7595415]


### Model Prediction on 1st Examples in Test Dataset

In [139]:
prediction([X_test[0]], w, b).numpy()[0][0]

5.5190086

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Data/11_Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [138]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
iris = pd.get_dummies(iris)

In [142]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X_iris = iris.drop(['Id','Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica'],axis=1)
y_iris = iris.iloc[:,-3:]

In [152]:
X_iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [153]:
y_iris.head()

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X_iris, y_iris, test_size=0.10 , random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [156]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax',input_shape=(4,)))

#Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


### Model Training 

In [160]:
model.fit(X_train_i, y_train_i, 
          validation_data=(X_test_i, y_test_i), 
          epochs=100,
          batch_size=X_train_i.shape[0])

Train on 135 samples, validate on 15 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
135/135 [==============================] - 0s 957us/sample - loss: 6.7937 - acc: 0.3259 - val_loss: 5.9007 - val_acc: 0.4000
Epoch 2/100
135/135 [==============================] - 0s 68us/sample - loss: 6.3955 - acc: 0.3259 - val_loss: 5.5234 - val_acc: 0.4000
Epoch 3/100
135/135 [==============================] - 0s 64us/sample - loss: 5.9894 - acc: 0.3259 - val_loss: 5.1468 - val_acc: 0.4000
Epoch 4/100
135/135 [==============================] - 0s 64us/sample - loss: 5.5844 - acc: 0.3259 - val_loss: 4.7717 - val_acc: 0.4000
Epoch 5/100
135/135 [==============================] - 0s 54us/sample - loss: 5.1810 - acc: 0.3259 - val_loss: 4.3989 - val_acc: 0.4000
Epoch 6/100
135/135 [==============================] - 0s 64us/sample - loss: 4.7804 - acc: 0.3259 - val_loss: 4.0299 - val_acc: 0.4000
Epoch 7/100
135/135 [==============================] - 0s 68us/sample - loss: 4.3842 - acc

### Model Prediction

In [163]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [178]:
#Actual Labels for the in test dataset
y_test_i

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
14,1,0,0
98,0,1,0
75,0,1,0
16,1,0,0
131,0,0,1
56,0,1,0
141,0,0,1
44,1,0,0
29,1,0,0
120,0,0,1


In [180]:
#Model prediction
prediction = model.predict(X_test_i)

#Print prediction of the test values
print(prediction)

[[0.49377778 0.20616877 0.30005348]
 [0.26121902 0.29579082 0.4429902 ]
 [0.25136244 0.31707564 0.43156195]
 [0.40872332 0.24159433 0.3496824 ]
 [0.16212012 0.4201559  0.41772398]
 [0.18449534 0.38236144 0.4331432 ]
 [0.09945554 0.3027949  0.59774953]
 [0.3963652  0.29716924 0.30646554]
 [0.4617893  0.2584624  0.27974823]
 [0.10068242 0.36571458 0.533603  ]
 [0.2355017  0.36347774 0.40102056]
 [0.411204   0.2667894  0.32200655]
 [0.13822044 0.37257165 0.48920798]
 [0.2120861  0.35308948 0.4348245 ]
 [0.20416762 0.3643114  0.43152094]]


### Save the Model

In [198]:
model.save('R6_Internallab_m1.h5')

RuntimeError: ignored

In [199]:
!ls -l

total 1628
-rw-r--r-- 1 root root 1535792 Mar 31 07:11 mnist_dnn_v1.h5
-rw-r--r-- 1 root root  104624 Mar 31 05:17 mnist_lc.h5
-rw-r--r-- 1 root root   16624 Mar 31 11:13 R6_Internallab_m1.h5
drwxr-xr-x 1 root root    4096 Mar 27 20:26 sample_data


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [0]:
#Initialize Sequential model
model2 = tf.keras.models.Sequential()

In [0]:
#Add 1st hidden layer
model2.add(tf.keras.layers.Dense(200, activation='sigmoid',input_shape=(4,)))

In [0]:
#Add 2nd hidden layer
model2.add(tf.keras.layers.Dense(200, activation='sigmoid'))

In [0]:
#Add Dense Layer which provides 3 Outputs after applying softmax
model2.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
#Compile the model
model2.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [195]:
model2.fit(X_train_i, y_train_i, 
          validation_data=(X_test_i, y_test_i), 
          epochs=100,
          batch_size=X_train_i.shape[0])

Train on 135 samples, validate on 15 samples
Epoch 1/100
135/135 [==============================] - 0s 1ms/sample - loss: 1.1832 - acc: 0.3333 - val_loss: 1.2181 - val_acc: 0.3333
Epoch 2/100
135/135 [==============================] - 0s 115us/sample - loss: 1.1580 - acc: 0.3333 - val_loss: 1.1904 - val_acc: 0.3333
Epoch 3/100
135/135 [==============================] - 0s 106us/sample - loss: 1.1388 - acc: 0.3333 - val_loss: 1.1684 - val_acc: 0.3333
Epoch 4/100
135/135 [==============================] - 0s 77us/sample - loss: 1.1242 - acc: 0.3333 - val_loss: 1.1512 - val_acc: 0.3333
Epoch 5/100
135/135 [==============================] - 0s 101us/sample - loss: 1.1135 - acc: 0.3630 - val_loss: 1.1378 - val_acc: 0.6000
Epoch 6/100
135/135 [==============================] - 0s 63us/sample - loss: 1.1057 - acc: 0.4963 - val_loss: 1.1274 - val_acc: 0.6000
Epoch 7/100
135/135 [==============================] - 0s 67us/sample - loss: 1.1001 - acc: 0.6370 - val_loss: 1.1194 - val_acc: 0.6000
E

In [196]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 200)               1000      
_________________________________________________________________
dense_5 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 603       
Total params: 41,803
Trainable params: 41,803
Non-trainable params: 0
_________________________________________________________________


In [197]:
#Model prediction
prediction = model2.predict(X_test_i)

#Print prediction of the test values
print(prediction)

[[0.3501666  0.3247245  0.32510892]
 [0.33740103 0.32418087 0.33841813]
 [0.32509437 0.32659402 0.34831157]
 [0.3502397  0.32469356 0.32506675]
 [0.31303298 0.3291744  0.35779262]
 [0.32439536 0.32771194 0.3478927 ]
 [0.3185725  0.32619104 0.35523644]
 [0.34775186 0.32595828 0.32628986]
 [0.35114628 0.3240954  0.3247583 ]
 [0.31631437 0.32728836 0.35639727]
 [0.32907444 0.32642734 0.34449822]
 [0.34789318 0.32560515 0.32650167]
 [0.3153312  0.32733706 0.35733178]
 [0.3251808  0.32722726 0.34759194]
 [0.32598466 0.3268361  0.34717923]]
